In [1]:
"""===============================================================
This is Sage code for Algorithm 1 in 'Markoff triples and Nielsen equivalence in SL_2(F_p)'. It tests whether the McCullough-Wanderley conjectures hold for primes not congruent to +/-1 mod 2d for a given input 'd'. Note that the paper suggests executing line 11 in Algorithm 1's pseudocode using Sage's "gcdx(f,g)" to find an integer element of the ideal (f,g) in Z[x]. It turns out to be much faster to use the "f.resultant(g)". With this modification, here are the runtimes using an AMD Ryzen Threadripper Pro (though there is no parallel programming in this code, so the processor may not make a huge difference): d=5 @ 6.5sec; d=4 @ 26sec; d=7 @ 1min 26sec; d=11 @ 1hr 8min; d=9 @ 1hr 16min; d=8 @ 2hr 23min; d=13 @ 5hr 21min; d=17 @ 66hr 39min. (The modulus in Theorem 1 from the paper can now be replaced with lcm(1,2,...,17).)   
==============================================================="""

from sage.arith.all import divisors, euler_phi
from sage.functions.other import ceil

"""===============================================================
Get user input for 'd'.
==============================================================="""

while True:
    try:
        d = int(input("\nEnter prime power d >= 4 to test the McCullough-Wanderley conjectures for primes not congruent to +/-1 mod 2d: "))
        if d >= 4 and is_prime_power(d):
            break
        else:
            print("Try again. ")
    except ValueError:
        print("Try again. ")

p,_ = is_prime_power(d, get_data=True)
if p == 2:
    n_d = 6 * d
elif p == 3:
    n_d = 5 * d
else:
    n_d = 4 * d

print(f"\nVerifying McCullough-Wanderley conjectures for all primes p > {2 * n_d} not congruent to +/-1 mod {2 * d}.")

"""===============================================================
Determine bounds denoted m_{d,n} in the paper and store in the vector called 'm_dn_all'. Also compute polynomials g_{d,n} in the paper (as described using Chebyshev polynomials) and store in the vector called 'g_dn_all'.
==============================================================="""

R.<x,y,z,kappa> = PolynomialRing(ZZ)
m_dn_all = []
g_dn_all = []
for n in range(d,n_d + 1, d):
    m_dn = ceil(sum(euler_phi(2 * n // i) for i in divisors(2 * n // (d * gcd(2,p)))) / 4)
    g_dn = R(chebyshev_U(n * d // p**(valuation(n, p) + 1) - 1, x / 2))
    if g_dn.degree(x) % 2 != 0:
        g_dn = x * g_dn
    m_dn_all.append(m_dn)
    g_dn_all.append(g_dn)
    n = n + d

max_deg = 0
for i in range(len(m_dn_all)):
    max_deg = max(4 * n_d + g_dn_all[i].degree(x) + 2 * m_dn_all[i], max_deg)

"""===============================================================
Precompute reductions of the form Phi(x^{2a}y^{2b}) for a + b <= 'max_deg' sufficiently large. Store them in 'reduction_cache'. During computation, the element reduction_cache[n][b][c] is Phi(x^{n-b-c}y^bz^c), which belongs to Z[x,kappa]. After computations, all reductions with nonzero 'c' or odd values of 'b' or 'n' are deleted.
==============================================================="""

print(f"\nPrecomputing reductions of monomials of the form x^a * y^b with b <= a and a + b <= {max_deg}: ", end='', flush=True)

reduction_cache = [] 
def reduce_monomial(n, b, c):
    a = n - b - c
    if b == 0 and c == 0:
        reduction_cache.append([])
        reduction_cache[n].append([])
        reduction_cache[n][0].append(x**a)
    elif c == 0 and b > 0:
        triple = sorted([a-1, b-1, 1], reverse=True)
        reduction_cache[n].append([])
        reduction_cache[n][b].append(2 * reduction_cache[n-1][triple[1]][triple[2]])
    else:
        triples = [sorted([a-1, b+1, c-1], reverse=True),sorted([a-1, b-1, c+1], reverse=True)]
        reduction_cache[n][b].append(reduction_cache[n-1][b-1][c-1] + reduction_cache[n-1][triples[0][1]][triples[0][2]] + reduction_cache[n-1][triples[1][1]][triples[1][2]] - kappa * reduction_cache[n-3][b-1][c-1])

for n in range(max_deg+1):
    for b in range(n//2 + 1):
        for c in range(min(b, n - 2*b) + 1):
            _ = reduce_monomial(n, b, c)
    if n >= 3:
        for b in range((n-3)//2 + 1):
            reduction_cache[n-3][b] = reduction_cache[n-3][b][0]
    if n == max_deg:
        print(n)
    else:
        print(f"{n}, ", end='', flush=True)
for n in range(max_deg-2,max_deg+1):
    for b in range(n//2 + 1):
        reduction_cache[n][b] = reduction_cache[n][b][0]
for n in reversed(range(len(reduction_cache))):
    if n % 2 != 0:
        reduction_cache.pop(n)
for row in reduction_cache:
    for n in reversed(range(len(row))):
        if n % 2 != 0:
            row.pop(n)

"""===============================================================
Now compute Phi(x^{2m}g_{d,n}f_n) for all n <= n_d and all m <= m_{d,n} using the building block reductions from reduction_cache. Discard the coefficients of x^0, x^2, and x^4 (because only rank n_d - 2 is required by Theorem 7.1), and store the remaining coefficients as a matrix row. 
==============================================================="""

def reduce_poly(poly):
    if poly == 0:
        return 0
    poly_divs = [(kappa - 4, None),(kappa - 2, 1),(kappa**2 - 5*kappa + 5,1),(kappa - 3, 2)]
    for poly_div, max_exp in poly_divs:
        count = 0
        while poly % poly_div == 0 and (max_exp is None or count < max_exp):
            poly = poly // poly_div
            count += 1
    return poly

def reduce_int(content):
    if content == 0:
        return 0
    for p in prime_range(2 * n_d):
        content = content // p**valuation(content, p)
    return content 

S.<kappa> = PolynomialRing(ZZ)
print(f"\n\nComputing reductions of {sum(m_dn_all)} eigenvectors: ", end='', flush=True)
rows = []
tot = 0
for n in range(d,n_d + 1, d):
    poly = 0
    for i in range(n+1):
        poly += ((-1)**i * 2 * n * binomial(n + i, n - i) // (n + i)) * (x**2 - kappa)**(n - i) * (x**2 - 4)**i * y**(2*i)
    poly *= g_dn_all[n // d - 1]
    for m in range(m_dn_all[n // d - 1]):
        output = 0
        for mo, co in zip(poly.monomials(), poly.coefficients()):
            exps = list(mo.exponents()[0])
            exps[0] = exps[0]//2 + m
            exps[1] = exps[1]//2
            output += co * kappa**exps[3] * reduction_cache[exps[0]+exps[1]][min(exps[0],exps[1])]
        content = output.content()
        output = reduce_int(content) * output // content
        rows.append([S(output.coefficient({x: i})) for i in range(6, 2 * n_d + 1, 2)])
        tot += 1
        print(f"{tot}, ", end='', flush=True)
M = Matrix(S, rows[::-1])
row_count = M.nrows()
col_count = n_d - 2
M_square = M.submatrix(0, 0, col_count, col_count)

print(f"\n\nMatrix of eigenvectors computed. Now computing the GCD of determinants of at most {lcm(col_count,row_count - col_count)} (hopefully only 3) principle minors: ", end='', flush=True)

"""===============================================================
The matrix M has several more rows than columns. To check if it has full column rank, compute gcds (denoted 'gcd_all') of maximal minor determinants, hoping to eventually get 1. As we go, we divide out primes from the gcd that are less than 2n_d because we aren't trying to prove full rank for those anyway, and we divide out powers of kappa-4, kappa-2, kappa-3, and kappa-5*kappa+5 as per Proposition 8.2. Also, there are tons of maximal minors to choose from, and their determinants can be computed in any order...however, the order selected below appears to always result in gcd_all = 1 after only one or two determinant computations, allowing for early loop termination.
==============================================================="""

poly_det = M_square.det()
content = poly_det.content()
poly_dets = [[reduce_int(content),reduce_poly(poly_det//content)]]
tot = 1;
print(f"{tot}, ", end='', flush=True)
gcd_all = 0
for replace_row in range(lcm(col_count,row_count - col_count)):
    M_copy = Matrix(M_square)
    M_copy[replace_row % col_count, :] = M[(replace_row % (row_count - col_count)) + col_count, :]
    poly_det = M_copy.det()
    content = poly_det.content()
    poly_det = reduce_poly(poly_det//content)
    content = reduce_int(content)
    for i in range(len(poly_dets)):
        result = lcm(content,poly_dets[i][0]) * reduce_int(poly_det.resultant(poly_dets[i][1]))
        gcd_all = gcd(gcd_all,result)
        if gcd_all == 1:
            break
    tot = tot + 1
    print(f"{tot}, ", end='', flush=True)
    if gcd_all == 1:
        break
    poly_dets.append([content,poly_det])

"""=============================================================== 
Success is only guaranteed for those primes that do not divide gcd_all and are not +/-1 mod 2d.
==============================================================="""

Try again. 
Try again. 

Verifying McCullough-Wanderley conjectures for all primes p > 40 not congruent to +/-1 mod 10.

Precomputing reductions of monomials of the form x^a * y^b with b <= a and a + b <= 100: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100


Computing reductions of 20 eigenvectors: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

Matrix of eigenvectors computed. Now computing the GCD of determinants of at most 18 (hopefully only 3) principle minors: 1, 2, 3, 

'=============================================================== \nSuccess is only guaranteed for those primes that do not divide gcd_all and are not +/-1 mod 2d.\n==============================================================='

above is Daniel's code 


my code is end in reduction algorism. (which is we talk about before). other things we need to check following chapter of paper (at least I don't understand)

In [20]:
from sage.arith.all import divisors, euler_phi
from sage.functions.other import ceil

# since we want d = 5 we can just ignore the initial coding about input & calculate p, n_d
d = 5
p = 5

n_d = 4 * d

# i don't know what following things about m_dn, g_dn and max_deg tho. So maybe we can reduce this code like make a easier m_dn or g_dn. not sure if that is possible
R.<x,y,z> = PolynomialRing(ZZ)
m_dn_all = [] # m_dn_all = [2, 4, 6, 8]
g_dn_all = [] # g_dn_all = [1, x^2, x^2 - 1, x^4 - 2*x^2]
for n in range(d,n_d + 1, d):
    m_dn = ceil(sum(euler_phi(2 * n // i) for i in divisors(2 * n // (d * gcd(2,p)))) / 4)
    g_dn = R(chebyshev_U(n * d // p**(valuation(n, p) + 1) - 1, x / 2))
    if g_dn.degree(x) % 2 != 0:
        g_dn = x * g_dn
    m_dn_all.append(m_dn)
    g_dn_all.append(g_dn)
    n = n + d

max_deg = 0 # max_deg = 100
for i in range(len(m_dn_all)):
    max_deg = max(4 * n_d + g_dn_all[i].degree(x) + 2 * m_dn_all[i], max_deg)

reduction_cache = [] 

# this one is store the function in reduction_cache [n][b]. 
# for example x^ay^b -> 2x^(a-1)y^(b-1)z, so store 2*reduction_cache[n-1][b][c] (because get -1 degree) in reduction_cache[n][b][0] (just a little bit like recurcive, so endedup store sth of pure x)
# I just delate the term with k, remain rest the same
def reduce_monomial(n, b, c): # x^(n-b-c), y^(b), z^(c)
    a = n - b - c
    if b == 0 and c == 0: # for b,c = 0 so base case (only have x)
        reduction_cache.append([])
        reduction_cache[n].append([])
        reduction_cache[n][0].append(x**a)
    elif c == 0 and b > 0: # for c = 0, using x^ay^b -> 2x^(a-1)y^(b-1)z
        triple = sorted([a-1, b-1, 1], reverse=True)
        reduction_cache[n].append([])
        reduction_cache[n][b].append(2 * reduction_cache[n-1][triple[1]][triple[2]])
    else: # for all things not equal to 0, using x^ay^bz^c = x^(a-1)y^(b-1)z^(c-1)(x^2+y^2+z^2)
        triples = [sorted([a-1, b+1, c-1], reverse=True),sorted([a-1, b-1, c+1], reverse=True)]
        reduction_cache[n][b].append(reduction_cache[n-1][b-1][c-1] + reduction_cache[n-1][triples[0][1]][triples[0][2]] + reduction_cache[n-1][triples[1][1]][triples[1][2]])

# Following is to fill the reduction_cache
for n in range(max_deg+1):
    for b in range(n//2 + 1):
        for c in range(min(b, n - 2*b) + 1):
            reduce_monomial(n, b, c)
    if n >= 3: # only want c = 0. not sure why he can do that.
        for b in range((n-3)//2 + 1):
            reduction_cache[n-3][b] = reduction_cache[n-3][b][0]
    # this following just check if every degree is filled 
    if n == max_deg:
        print(n)
    else:
        print(f"{n}, ", end='', flush=True)


# check if every base case if there
for n in range(len(reduction_cache)):
    val = reduction_cache[n][0]   # b = 0 c = 0
    print(n, val)

# check specific function (I just check some it seems like okey)
n = 8
b = 2
c = 1      # x^0 y^0 z^2

print(reduction_cache[n][b])

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
0 1
1 x
2 x^2
3 x^3
4 x^4
5 x^5
6 x^6
7 x^7
8 x^8
9 x^9
10 x^10
11 x^11
12 x^12
13 x^13
14 x^14
15 x^15
16 x^16
17 x^17
18 x^18
19 x^19
20 x^20
21 x^21
22 x^22
23 x^23
24 x^24
25 x^25
26 x^26
27 x^27
28 x^28
29 x^29
30 x^30
31 x^31
32 x^32
33 x^33
34 x^34
35 x^35
36 x^36
37 x^37
38 x^38
39 x^39
40 x^40
41 x^41
42 x^42
43 x^43
44 x^44
45 x^45
46 x^46
47 x^47
48 x^48
49 x^49
50 x^50
51 x^51
52 x^52
53 x^53
54 x^54
55 x^55
56 x^56
57 x^57
58 x^58
59 x^59
60 x^60
61 x^61
62 x^62
63 x^63
64 x^64
65 x^65
66 x^66
67 x^67
68 x^68
69 x^69
70 x^70
71 x^71
72 x^72
73 x^73
74 x^74
75 x^75
76 x^76
77 x^77
78 x^7